In [1]:
import jax
import jax.numpy as jnp

In [2]:
key = jax.random.PRNGKey(42)

In [3]:
%time key = jax.random.PRNGKey(42)

CPU times: user 2.1 ms, sys: 0 ns, total: 2.1 ms
Wall time: 1.65 ms


In [4]:
%time jax.random.uniform(key, shape=(1000*16*16*16, 3), minval=1. - 0.3, maxval=1. + 0.3)

CPU times: user 632 ms, sys: 63.4 ms, total: 695 ms
Wall time: 222 ms


DeviceArray([[0.8752752 , 1.2344012 , 1.1507926 ],
             [1.2555599 , 1.2922567 , 0.70989543],
             [1.0282397 , 0.9810415 , 0.9582528 ],
             ...,
             [0.91074455, 0.91265196, 1.2724572 ],
             [1.2734149 , 1.1726161 , 1.2775979 ],
             [0.9419782 , 1.2352306 , 0.96576226]], dtype=float32)

In [5]:
%%time
key, subkey = jax.random.split(key)
jax.random.uniform(subkey, shape=(1000*16*16*16, 3), minval=1. - 0.3, maxval=1. + 0.3)

CPU times: user 545 ms, sys: 53.3 ms, total: 599 ms
Wall time: 148 ms


DeviceArray([[1.2910093 , 1.0656897 , 0.7684896 ],
             [1.1905303 , 1.222494  , 0.83090544],
             [0.98227715, 0.71771586, 1.017778  ],
             ...,
             [0.89847183, 0.882926  , 1.1704695 ],
             [1.0654551 , 0.99314874, 1.0599623 ],
             [0.82162833, 1.2348583 , 0.8477738 ]], dtype=float32)

## Conclusion: `jax.random.split` is a lightweight operation

In [6]:
import nndt
from nndt.space2 import *

In [7]:
!ls ../tests/

acdc_for_test			 test_import.py
acdc_for_test.7z		 test_load_datasets.py
BROKEN_test_space2_OLD.py	 test_math_core.py
BROKEN_test_space2_plot_tree.py  test_space2_basic.py
__init__.py			 test_space2_loaders.py
__pycache__			 test_space2_method_set.py
result.vtp			 test_space2_primitive_sdf.py
test_file.png			 test_space2_transformation.py
test_folder_tree		 test_space_model.py
test_haiku_modules.py		 test_vizualize.py


In [8]:
space = load_from_path('../tests/acdc_for_test')
space.preload()
print(space.print('default'))

S:space 0.0.2a4
├── O3D:patient009 ((0.00, 0.00, 0.00), (134.00, 183.00, 84.00))
│   ├── FS:colored_obj mesh_obj^ ../tests/acdc_for_test/patient009/colored.obj
│   ├── FS:sdf_npy sdt^ ../tests/acdc_for_test/patient009/sdf.npy
│   ├── TR:transform identity
│   ├── MS:mesh
│   ├── MS:sampling
│   ├── MS:sdt
│   ├── plot(default, filepath=None)
│   ├── print(default|source|full)
│   ├── sampling_eachN_from_mesh(count=N, step=M, shift=K) -> (ns_ind[N], ns_xyz[N])
│   ├── sampling_grid(spacing=(D,H,W)) -> ns_xyz[D,H,W,3]
│   ├── sampling_grid_with_noise(key, spacing=(D,H,W), sigma) -> ns_xyz[N,3]
│   ├── sampling_uniform(key, N) -> ns_xyz[N,3]
│   ├── save_mesh(filepath, {name, array})
│   ├── surface_ind2xyz(ns_ind[...,1]) -> ns_xyz[...,3]
│   ├── surface_xyz2ind(ns_xyz[...,3]) -> ns_ind[...,1]
│   ├── surface_xyz2sdt(ns_xyz[...,3]) -> ns_sdt[...,1]
│   ├── transform_sdt_ns2ps(ns_sdt[...]) -> ps_sdt[...]
│   ├── transform_sdt_ps2ns(ps_sdt[...]) -> ns_sdt[...]
│   ├── transform_xyz_ns2ps(ns

In [9]:
xyz = jax.random.uniform(subkey, 
                   shape=(1000, 16 ,16 ,16, 3), 
                   minval=1. - 0.3, maxval=1. + 0.3)

scale = jax.random.uniform(subkey, 
                   shape=(1000, 3), 
                   minval=1. - 0.3, maxval=1. + 0.3)

rotate = jax.random.uniform(subkey, 
                   shape=(1000, 3), 
                   minval=0. - 0.3, maxval=0. + 0.3)

In [10]:
%%time
space.patient009.surface_xyz2sdt(xyz)

CPU times: user 302 ms, sys: 122 ms, total: 424 ms
Wall time: 302 ms


DeviceArray([[[[[139.15819],
                [139.15819],
                [140.44928],
                ...,
                [140.44928],
                [140.5169 ],
                [140.02856]],

               [[139.15819],
                [140.02856],
                [139.64957],
                ...,
                [140.44928],
                [139.64957],
                [139.0899 ]],

               [[139.15819],
                [140.44928],
                [139.58151],
                ...,
                [140.44928],
                [139.0899 ],
                [139.58151]],

               ...,

               [[139.0899 ],
                [140.44928],
                [139.0899 ],
                ...,
                [139.96071],
                [140.5169 ],
                [140.02856]],

               [[140.5169 ],
                [139.58151],
                [140.44928],
                ...,
                [140.02856],
                [139.58151],
                [139.0899

In [11]:
%%time
space.patient009.surface_xyz2sdt(xyz.reshape((-1,3)))

CPU times: user 121 ms, sys: 105 ms, total: 226 ms
Wall time: 98.4 ms


DeviceArray([[139.15819],
             [139.15819],
             [140.44928],
             ...,
             [140.44928],
             [139.96071],
             [139.64957]], dtype=float32)

In [12]:
comp_space = jax.jit(space.patient009.surface_xyz2sdt)
comp_space(xyz.reshape((-1,3)))

DeviceArray([[139.15819],
             [139.15819],
             [140.44928],
             ...,
             [140.44928],
             [139.96071],
             [139.64957]], dtype=float32)

In [13]:
%%time
comp_space(xyz.reshape((-1,3)))

CPU times: user 43.7 ms, sys: 0 ns, total: 43.7 ms
Wall time: 7.28 ms


DeviceArray([[139.15819],
             [139.15819],
             [140.44928],
             ...,
             [140.44928],
             [139.96071],
             [139.64957]], dtype=float32)

## Conclusion: jax.jit improves performance here!

In [14]:
def scale_xyz(xyz, scale=(1., 1., 1.)):
    return scale*xyz

vec_scale_xyz = jax.vmap(scale_xyz, (0,0))

In [15]:
%%time
vec_scale_xyz(xyz, scale).shape

CPU times: user 42.3 ms, sys: 0 ns, total: 42.3 ms
Wall time: 25.2 ms


(1000, 16, 16, 16, 3)

In [16]:
comp_vec_scale_xyz = jax.jit(vec_scale_xyz)
comp_vec_scale_xyz(xyz, scale);

In [17]:
%%time
comp_vec_scale_xyz(xyz, scale).shape

CPU times: user 27 ms, sys: 713 µs, total: 27.7 ms
Wall time: 7.54 ms


(1000, 16, 16, 16, 3)

## Conclusion: jax.jit improve performance here!

In [18]:
xyz.shape

(1000, 16, 16, 16, 3)

In [19]:
(xyz @ jnp.eye(3)).shape

(1000, 16, 16, 16, 3)

In [20]:
xyz.shape, xyz.T.shape

((1000, 16, 16, 16, 3), (3, 16, 16, 16, 1000))

In [21]:
def rotation_matrix(yaw, pitch, roll):
    Rz = jnp.array([[jnp.cos(yaw), -jnp.sin(yaw), 0.],
                    [jnp.sin(yaw), jnp.cos(yaw), 0.],
                    [0., 0., 1.]])
    Ry = jnp.array([[jnp.cos(pitch), 0, jnp.sin(pitch)],
                    [0, 1, 0],
                    [-jnp.sin(pitch), 0., jnp.cos(pitch)]])
    Rx = jnp.array([[1., 0., 0.],
                    [0., jnp.cos(roll), -jnp.sin(roll)],
                    [0., jnp.sin(roll), jnp.cos(roll)]])

    return Rz @ Ry @ Rx

In [22]:
def rotate_xyz(xyz, M=(1., 1., 1.)):
    M = rotation_matrix(M[0], M[1], M[2]).T
    return xyz @ M

vec_rotate_xyz = jax.vmap(rotate_xyz, (0,0))

In [23]:
%%time
vec_rotate_xyz(xyz, rotate).shape

CPU times: user 331 ms, sys: 10.9 ms, total: 342 ms
Wall time: 340 ms


(1000, 16, 16, 16, 3)

In [24]:
comp_vec_rotate_xyz = jax.jit(vec_scale_xyz)
comp_vec_rotate_xyz(xyz, rotate);

In [25]:
%%time
comp_vec_rotate_xyz(xyz, scale).shape

CPU times: user 22 ms, sys: 0 ns, total: 22 ms
Wall time: 6.09 ms


(1000, 16, 16, 16, 3)

## Conclusion: jax.jit improve performance here!

In [55]:
samp_ = space.patient009.sampling_eachN_from_mesh

In [77]:
N = 10000
SIGMA = 0.02

In [78]:
%%time
mesh_ind, mesh_xyz = samp_(N, 77, 3);

CPU times: user 2.36 ms, sys: 281 µs, total: 2.64 ms
Wall time: 1.59 ms


In [79]:
mesh_ind.shape, mesh_xyz.shape

((10000,), (10000, 3))

In [80]:
# This code MUST be separated, 
# because parameters that effects output size must be marked and causes recompilation of jit on each call!
xyz_cube = jax.random.uniform(subkey, 
                   shape=(N, 16 ,16 ,16, 3), 
                   minval=1. - 0.3, maxval=1. + 0.3)

scale = jax.random.uniform(subkey, 
                   shape=(N, 3), 
                   minval=1. - 0.3, maxval=1. + 0.3)

rotate = jax.random.uniform(subkey, 
                   shape=(N, 3), 
                   minval=0. - 0.3, maxval=0. + 0.3)

shift = jax.random.normal(subkey, shape=(N, 3))*SIGMA

xyz_cube.shape, scale.shape, rotate.shape, shift.shape

((10000, 16, 16, 16, 3), (10000, 3), (10000, 3), (10000, 3))

In [81]:
def rotation_matrix(yaw, pitch, roll):
    Rz = jnp.array([[jnp.cos(yaw), -jnp.sin(yaw), 0.],
                    [jnp.sin(yaw), jnp.cos(yaw), 0.],
                    [0., 0., 1.]])
    Ry = jnp.array([[jnp.cos(pitch), 0, jnp.sin(pitch)],
                    [0, 1, 0],
                    [-jnp.sin(pitch), 0., jnp.cos(pitch)]])
    Rx = jnp.array([[1., 0., 0.],
                    [0., jnp.cos(roll), -jnp.sin(roll)],
                    [0., jnp.sin(roll), jnp.cos(roll)]])

    return Rz @ Ry @ Rx

def scale_xyz(cube, scale=jnp.array([1., 1., 1.])):
    return scale*cube

def rotate_xyz(cube, M=jnp.array([1., 1., 1.])):
    M = rotation_matrix(M[0], M[1], M[2]).T
    return cube @ M

def shift_xyz(cube, shift=jnp.array([0., 0., 0.])):
    return cube + shift

def scale_rotate_shift(cube, scale=(1., 1., 1.), M=(0., 0., 0.), shift=(0.,0.,0.)):
    cube = scale_xyz(cube)
    cube = rotate_xyz(cube, M=M)
    cube = shift_xyz(cube, shift=shift)
    return cube

vec_scale_rotate_shift = jax.jit(jax.vmap(scale_rotate_shift, (0,0,0,0)))
vec_scale_rotate_shift(xyz_cube, scale, rotate, shift) 

DeviceArray([[[[[1.137713  , 0.61757845, 1.5118213 ],
                [0.6667725 , 0.5031054 , 1.5550908 ],
                [0.7821145 , 0.80558527, 1.3547583 ],
                ...,
                [0.96897566, 0.32650533, 1.0141561 ],
                [0.8080453 , 0.7234298 , 1.2506273 ],
                [0.8273762 , 0.7757545 , 1.1804826 ]],

               [[0.94744384, 0.60338825, 1.6180462 ],
                [1.0729871 , 0.12411419, 1.4759398 ],
                [0.79832137, 0.3934863 , 1.331535  ],
                ...,
                [0.6528391 , 0.45543724, 1.5131797 ],
                [0.8117471 , 0.5495241 , 1.2185587 ],
                [1.2474134 , 0.6897423 , 1.366688  ]],

               [[1.2070967 , 0.5832151 , 1.5714577 ],
                [0.6509837 , 0.30136293, 1.4875851 ],
                [1.1334618 , 0.36888936, 1.287113  ],
                ...,
                [0.92089796, 0.68863535, 1.6325104 ],
                [0.7119235 , 0.5966444 , 1.4701604 ],
               

In [82]:
%%time
cubes = vec_scale_rotate_shift(xyz_cube, scale, rotate, shift)

CPU times: user 291 ms, sys: 190 ms, total: 480 ms
Wall time: 259 ms


In [83]:
%%time
comp_space(cubes)

CPU times: user 445 ms, sys: 47.1 ms, total: 492 ms
Wall time: 43.2 ms


DeviceArray([[[[[139.96071],
                [140.44928],
                [140.44928],
                ...,
                [140.44928],
                [140.44928],
                [140.44928]],

               [[140.44928],
                [139.96071],
                [140.44928],
                ...,
                [140.44928],
                [140.44928],
                [139.96071]],

               [[139.96071],
                [140.44928],
                [139.96071],
                ...,
                [140.44928],
                [140.44928],
                [139.96071]],

               ...,

               [[139.96071],
                [139.96071],
                [139.96071],
                ...,
                [139.96071],
                [140.44928],
                [140.44928]],

               [[139.96071],
                [139.96071],
                [139.96071],
                ...,
                [139.96071],
                [140.44928],
                [140.4492

In [52]:
mesh_xyz.shape

(10000, 3)